In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [2]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [3]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [4]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [5]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [6]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [7]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [8]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [9]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [10]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Some code is going to >48 hours to ran. Lucky it's highly parellalisable so we can use a compute cluster. The one option is local to split across CPU cores. The better option is to go remote and explote 32 cores on multiple AWS machines.

This simple python interface provide what we need to do that. Don't forget to port forward!. This allows the remote machine to connect to ports on the master via it's localhost loopback. 

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [12]:
#remote_host="ubuntu@184.73.51.120"
#!remote_pfwd ubuntu@52.90.20.45 {11305..11307}

In [11]:
class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, port_range=list(range(11305, 11340))):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(port_range), master="localhost", outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [12]:
cluster_provider = ClusterProvider(remote_hosts=[("ubuntu@54.204.196.107", 8)],
                                    port_range = list(range(11305, 11338)))
# cluster_provider = ClusterProvider(n_nodes=8)

In [23]:
cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [13]:
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [14]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    
    propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [15]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Add prop scores to covar data
    propensity_scores = np.array(get_propensity_scores(assignments, covariate_data))
    matching_data = np.hstack([covariate_data, propensity_scores.reshape(-1, 1)])
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=covariate_data,
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=covariate_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [16]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [17]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [18]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method):
    return method(outcomes, assignments, covar_data)

In [19]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [ ]:
# DEBUG:
run_simulation(runs=50, n_samples=1000, assignment_model="A_add_lin",
              estimator=genmatch_est, verbose=True)

GenMatch Time:  18.928613901138306
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  13.397344827651978
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  55.6703314781189
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  38.81633377075195
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.



### Run MC trial

In [20]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']
def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     models=assignment_models,
                     overwrite=False, verbosity=1):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage
    if store_name is None:
        if set(models) == set(assignment_model_names):
            store_name = "est_{}_runs_{}_n_{}".format(
                est.__name__,
                runs,
                n_samples)
        else:
            store_name = "est_{}_runs_{}_n_{}_models_{}".format(
                est.__name__,
                runs,
                n_samples,
                "_".join(models))
            
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in models:
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2))
            store_results_dict(results[model], store_name+"_checkpoint_"+model)
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

In [21]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.045874914703647685
1 RMSE:  0.07310500057973227 

Model:  B_add_mild_nlin
1 Bias:  3.1844355433209786
1 RMSE:  0.06588422028138122 

Model:  C_add_mod_nlin
1 Bias:  10.094350684204597
1 RMSE:  0.07650839711310455 

Model:  D_mild_nadd_lin
1 Bias:  6.720731771408928
1 RMSE:  0.08531717119502563 

Model:  E_mild_nadd_mild_nlin
1 Bias:  10.36168716658826
1 RMSE:  0.09094245826533698 

Model:  F_mod_nadd_lin
1 Bias:  3.1228082403965436
1 RMSE:  0.07605107262377982 

Model:  G_mod_nadd_mod_nlin
1 Bias:  11.830178367664905
1 RMSE:  0.07798212919046259 



In [22]:
run_test_battery(
    est=genmatch_est,
    runs=1000,
    n_samples=1000,
    models=assignment_model_names[3:5],
    verbosity=2)

No valid, existant results found. Beggining battery.

Running:  D_mild_nadd_lin


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  17.909931659698486
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  8.087615489959717
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  15.974268436431885
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  24.009857654571533
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  24.796778678894043
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  18.609666347503662
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the t

GenMatch Time:  21.197824954986572
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  15.027225255966187
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  17.809581518173218
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  24.58236074447632
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  14.612401962280273
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  34.56177258491516
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ti

GenMatch Time:  27.94295644760132
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  24.758838176727295
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

Done 100 of 1000
GenMatch Time:  20.11139488220215
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  15.98286747932434
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  13.952086448669434
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  22.2979154586792
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be fast

...

GenMatch Time:  18.87963318824768
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  17.869734287261963
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  14.975420236587524
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  28.558820009231567
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  9.560300588607788
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  24.745657205581665
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ti

GenMatch Time:  21.538252115249634
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  14.427932262420654
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  11.818673610687256
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  13.985768795013428
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  23.744250535964966
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  13.438432693481445
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the 

GenMatch Time:  16.33001160621643
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  13.833427667617798
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  32.70485305786133
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  15.24163818359375
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  17.71773338317871
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties=FALSE option.

GenMatch Time:  26.004523754119873
Increasing memory because of ties: allocating a matrix of size 3 times 200000 doubles.

I would be faster with the ties

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: There were 50 or more warnings (use warnings() to see the first 50)
  warnings.warn(x, RRuntimeWarning)


RRuntimeError: 

GenMatch bias on 1000/1000 = 4.26790791538966